In [2]:
import torchvision.datasets as dataset
import torchvision.transforms as transform
from torch.utils.data import DataLoader
from model_search import Network_Multi_Path as Network
import matplotlib.pyplot as plt
class Compose(object):
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, image, target):
        for t in self.transforms:
            image = t(image)
            target = t(target)
            target = target*255
            target = target.long()
        return image, target

train_dataset = dataset.Cityscapes('/datasets/cityscapes/',split='train', mode='fine',
                     target_type='semantic',transforms=Compose([transform.ToTensor()]))

train_dataloader = DataLoader(train_dataset, batch_size=12, shuffle=True, num_workers=4)

/home/jif055/model/utils/darts_utils.py:179: UserWarning: TensorRT (or pycuda) is not installed. compute_latency_ms_tensorrt() cannot be used.
  warnings.warn("TensorRT (or pycuda) is not installed. compute_latency_ms_tensorrt() cannot be used.")


In [4]:
imgs, labels = iter(train_dataloader).next()

In [3]:
model = Network()

In [5]:
output = model(imgs)

AssertionError: 
Found no NVIDIA driver on your system. Please check that you
have an NVIDIA GPU and installed a driver from
http://www.nvidia.com/Download/index.aspx